In [11]:
from keras.models import Sequential,  Model
from keras.layers import Convolution2D,Input,BatchNormalization,Conv2D,Activation,Lambda,Subtract,Conv2DTranspose, PReLU
from keras.regularizers import l2
from keras.layers import  Reshape,Dense,Flatten
# from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
from scipy.io import loadmat
import keras.backend as K
# from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
import numpy as np
import math
from scipy import interpolate
#from scipy.misc import imresize


def psnr(target, ref):
    # assume RGB image
    target_data = np.array(target, dtype=float)
    ref_data = np.array(ref, dtype=float)

    diff = ref_data - target_data
    diff = diff.flatten('C')

    rmse = math.sqrt(np.mean(diff ** 2.))

    return 20 * math.log10(255. / rmse)

def interpolation(noisy , SNR , Number_of_pilot , interp):
    noisy_image = np.zeros((40000,72,14,2))

    noisy_image[:,:,:,0] = np.real(noisy)
    noisy_image[:,:,:,1] = np.imag(noisy)


    if (Number_of_pilot == 48):
        idx = [14*i for i in range(1, 72,6)]+[4+14*(i) for i in range(4, 72,6)]+[7+14*(i) for i in range(1, 72,6)]+[11+14*(i) for i in range(4, 72,6)]
    elif (Number_of_pilot == 16):
        idx= [4+14*(i) for i in range(1, 72,9)]+[9+14*(i) for i in range(4, 72,9)]
    elif (Number_of_pilot == 24):
        idx = [14*i for i in range(1,72,9)]+ [6+14*i for i in range(4,72,9)]+ [11+14*i for i in range(1,72,9)]
    elif (Number_of_pilot == 8):
      idx = [4+14*(i) for  i in range(5,72,18)]+[9+14*(i) for i in range(8,72,18)]
    elif (Number_of_pilot == 36):
      idx = [14*(i) for  i in range(1,72,6)]+[6+14*(i) for i in range(4,72,6)] + [11+14*i for i in range(1,72,6)]



    r = [x//14 for x in idx]
    c = [x%14 for x in idx]



    interp_noisy = np.zeros((40000,72,14,2))

    for i in range(len(noisy)):
        z = [noisy_image[i,j,k,0] for j,k in zip(r,c)]
        if(interp == 'rbf'):
            f = interpolate.Rbf(np.array(r).astype(float), np.array(c).astype(float), z,function='gaussian')
            X , Y = np.meshgrid(range(72),range(14))
            z_intp = f(X, Y)
            interp_noisy[i,:,:,0] = z_intp.T
        elif(interp == 'spline'):
            tck = interpolate.bisplrep(np.array(r).astype(float), np.array(c).astype(float), z)
            z_intp = interpolate.bisplev(range(72),range(14),tck)
            interp_noisy[i,:,:,0] = z_intp
        z = [noisy_image[i,j,k,1] for j,k in zip(r,c)]
        if(interp == 'rbf'):
            f = interpolate.Rbf(np.array(r).astype(float), np.array(c).astype(float), z,function='gaussian')
            X , Y = np.meshgrid(range(72),range(14))
            z_intp = f(X, Y)
            interp_noisy[i,:,:,1] = z_intp.T
        elif(interp == 'spline'):
            tck = interpolate.bisplrep(np.array(r).astype(float), np.array(c).astype(float), z)
            z_intp = interpolate.bisplev(range(72),range(14),tck)
            interp_noisy[i,:,:,1] = z_intp


    interp_noisy = np.concatenate((interp_noisy[:,:,:,0], interp_noisy[:,:,:,1]), axis=0).reshape(80000, 72, 14, 1)
   
    
    return interp_noisy

def SRCNN_model():

    input_shape = (72,14,1)
    x = Input(shape = input_shape)
    c1 = Conv2D( 64 , (9 , 9) ,activation = 'relu', kernel_initializer = 'he_normal', padding='same')(x)
    c2 = Conv2D( 32 ,(1,1), activation = 'relu', kernel_initializer = 'he_normal', padding='same')(c1)
    c3 = Conv2D( 1 , (5,5), kernel_initializer = 'he_normal', padding='same')(c2)
    #c4 = Input(shape = input_shape)(c3)
    model = Model(inputs = x, outputs = c3)
    ##compile
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8) 
    model.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error']) 
    return model
    
def SRCNN_train(train_data ,train_label, val_data , val_label , channel_model, num_pilots , SNR ):
    srcnn_model = SRCNN_model()
    print(srcnn_model.summary())
    
    checkpoint = ModelCheckpoint("SRCNN_check.h5", monitor='val_loss', verbose=1, save_best_only=True,
                                 save_weights_only=False, mode='min')
    callbacks_list = [checkpoint]

    srcnn_model.fit(train_data, train_label, batch_size=128, validation_data=(val_data, val_label),
                    callbacks=callbacks_list, shuffle=True, epochs= 10 , verbose=1)
    
    #srcnn_model.save_weights("drive/codes/my_srcnn/SRCNN_SUI5_weights/SRCNN_48_12.h5")
    srcnn_model.save_weights("SRCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(SNR) + ".h5")
   


def SRCNN_predict(input_data , channel_model , num_pilots , SNR):
    srcnn_model = SRCNN_model()
    srcnn_model.load_weights("SRCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(SNR) + ".h5")
    predicted  = srcnn_model.predict(input_data)
    return predicted

  
def DNCNN_model ():
  
    inpt = Input(shape=(None,None,1))
    # 1st layer, Conv+relu
    x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(inpt)
    x = Activation('relu')(x)
    # 18 layers, Conv+BN+relu
    for i in range(18):
        x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
        x = BatchNormalization(axis=-1, epsilon=1e-3)(x)
        x = Activation('relu')(x)   
    # last layer, Conv
    x = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x = Subtract()([inpt, x])   # input - noise
    model = Model(inputs=inpt, outputs=x)
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8) 
    model.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])    
    return model

def DNCNN_train(train_data ,train_label, val_data , val_label, channel_model , num_pilots , SNR ):
  
  dncnn_model = DNCNN_model()
  print(dncnn_model.summary())

  checkpoint = ModelCheckpoint("DNCNN_check.h5", monitor='val_loss', verbose=1, save_best_only=True,
                               save_weights_only=False, mode='min')
  callbacks_list = [checkpoint]

  dncnn_model.fit(train_data, train_label, batch_size=128, validation_data=(val_data, val_label),
                  callbacks=callbacks_list, shuffle=True, epochs= 5 , verbose=1)
  dncnn_model.save_weights("DNCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(SNR) + ".h5")
  
  
  
def DNCNN_predict(input_data, channel_model , num_pilots , SNR):
  dncnn_model = DNCNN_model()
  dncnn_model.load_weights("DNCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(SNR) + ".h5")
  predicted  = dncnn_model.predict(input_data)
  return predicted
  
  


In [3]:
noisy_input = loadmat("My_noisy_H_22", appendmat = True )['My_noisy_H']

In [4]:
noisy_input[32][23][4]

(-0.012686757842631352-0.8107323370829541j)

In [5]:
type(noisy_input)
noisy_input.shape
#40000 channels 
#72 frequency subcarriers
#14 time slots

(40000, 72, 14)

In [6]:
perfect = loadmat("Perfect_H_40000", appendmat=True)['My_perfect_H']

In [7]:
perfect.shape

(40000, 72, 14)

In [21]:
# reshape to 2d
#mat = np.reshape(perfect[1],(256,256))
import numpy as np
from PIL import Image
mat = noisy_input[5600]
# Creates PIL image
img = Image.fromarray(np.uint8(mat * 255),'L')
#img = Image.fromarray(np.uint8(m , 'L')
img.show()


<ipython-input-21-7fc29c541147>:7: ComplexWarning: Casting complex values to real discards the imaginary part
  img = Image.fromarray(np.uint8(mat * 255),'L')


In [22]:
type(perfect)
perfect.shape


(40000, 72, 14)

In [8]:
#signal to noise ratio
def psnr(target, ref):
    # assume RGB image
    target_data = np.array(target, dtype=float)
    ref_data = np.array(ref, dtype=float)

    diff = ref_data - target_data
    diff = diff.flatten('C')

    rmse = math.sqrt(np.mean(diff ** 2.))

    return 20 * math.log10(255. / rmse)

In [9]:
def interpolation(noisy, SNR, Number_of_pilot, interp):
    noisy_image = np.zeros((40000,72,14,2))

    noisy_image[:,:,:,0] = np.real(noisy)
    noisy_image[:,:,:,1] = np.imag(noisy)


    if (Number_of_pilot == 48):
        idx = [14*i for i in range(1, 72,6)]+[4+14*(i) for i in range(4, 72,6)]+[7+14*(i) for i in range(1, 72,6)]+[11+14*(i) for i in range(4, 72,6)]
    elif (Number_of_pilot == 16):
        idx= [4+14*(i) for i in range(1, 72,9)]+[9+14*(i) for i in range(4, 72,9)]
    elif (Number_of_pilot == 24):
        idx = [14*i for i in range(1,72,9)]+ [6+14*i for i in range(4,72,9)]+ [11+14*i for i in range(1,72,9)]
    elif (Number_of_pilot == 8):
        idx = [4+14*(i) for  i in range(5,72,18)]+[9+14*(i) for i in range(8,72,18)]
    elif (Number_of_pilot == 36):
        idx = [14*(i) for  i in range(1,72,6)]+[6+14*(i) for i in range(4,72,6)] + [11+14*i for i in range(1,72,6)]



    r = [x//14 for x in idx]
    c = [x%14 for x in idx]



    interp_noisy = np.zeros((40000,72,14,2))

    for i in range(len(noisy)):
        z = [noisy_image[i,j,k,0] for j,k in zip(r,c)]
        if(interp == 'rbf'):
            f = interpolate.Rbf(np.array(r).astype(float), np.array(c).astype(float), z,function='gaussian')
            X , Y = np.meshgrid(range(72),range(14))
            z_intp = f(X, Y)
            interp_noisy[i,:,:,0] = z_intp.T
        elif(interp == 'spline'):
            tck = interpolate.bisplrep(np.array(r).astype(float), np.array(c).astype(float), z)
            z_intp = interpolate.bisplev(range(72),range(14),tck)
            interp_noisy[i,:,:,0] = z_intp
        z = [noisy_image[i,j,k,1] for j,k in zip(r,c)]
        if(interp == 'rbf'):
            f = interpolate.Rbf(np.array(r).astype(float), np.array(c).astype(float), z,function='gaussian')
            X , Y = np.meshgrid(range(72),range(14))
            z_intp = f(X, Y)
            interp_noisy[i,:,:,1] = z_intp.T
        elif(interp == 'spline'):
            tck = interpolate.bisplrep(np.array(r).astype(float), np.array(c).astype(float), z)
            z_intp = interpolate.bisplev(range(72),range(14),tck)
            interp_noisy[i,:,:,1] = z_intp


    interp_noisy = np.concatenate((interp_noisy[:,:,:,0], interp_noisy[:,:,:,1]), axis=0).reshape(80000, 72, 14, 1)
   
    
    return interp_noisy

In [22]:
###### import numpy as np
import numpy
import math
#from models import interpolation , SRCNN_train , SRCNN_model, SRCNN_predict , DNCNN_train , DNCNN_model , DNCNN_predict
#from scipy.misc import imresize
from scipy.io import loadmat
import matplotlib.pyplot as plt



if __name__ == "__main__":
    # load datasets 
    channel_model = "VehA"
    SNR = 22
    Number_of_pilots = 48
  #  perfect = loadmat("Perfect_H_40000"+ channel_model.mat)['My_perfect_H']
    #perfect = loadmat("Perfect_H_40000", appendmat = True)['My_perfect_H']
    #noisy_input = loadmat("Noisy_" + channel_model + "_" + "SNR_" + str(SNR) + ".mat") [channel_model+"_noisy_"+ str(SNR)]  
    #noisy_input = loadmat("My_noisy_H_22", appendmat = True )["My_noisy_H"]  
                      
    interp_noisy = interpolation(noisy_input , SNR , Number_of_pilots , 'rbf')

    perfect_image = numpy.zeros((len(perfect),72,14,2))
    perfect_image[:,:,:,0] = numpy.real(perfect)
    perfect_image[:,:,:,1] = numpy.imag(perfect)
    perfect_image = numpy.concatenate((perfect_image[:,:,:,0], perfect_image[:,:,:,1]), axis=0).reshape(2*len(perfect), 72, 14, 1)
    
    
    ####### ------ training SRCNN ------ #######
    idx_random = numpy.random.rand(len(perfect_image)) < (1/9)  # uses 32000 from 36000 as training and the rest as validation
    train_data, train_label = interp_noisy[idx_random,:,:,:] , perfect_image[idx_random,:,:,:]
    val_data, val_label = interp_noisy[~idx_random,:,:,:] , perfect_image[~idx_random,:,:,:]    
    SRCNN_train(train_data ,train_label, val_data , val_label , channel_model , Number_of_pilots , SNR )
    
   
    ####### ------ prediction using SRCNN ------ #######
    srcnn_pred_train = SRCNN_predict(train_data, channel_model , Number_of_pilots , SNR)
    srcnn_pred_validation = SRCNN_predict(val_data, channel_model , Number_of_pilots , SNR)  
                      
                      
    ####### ------ training DNCNN ------ #######
    
    DNCNN_train(train_data ,train_label, val_data , val_label, channel_model , Number_of_pilots, SNR )
    
    ######--------predicting using DNCNN ----####
    
    DNCNN_pred = DNCNN_predict(train_data, channel_model , Number_of_pilots , SNR)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 72, 14, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 72, 14, 64)        5248      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 14, 32)        2080      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 14, 1)         801       
Total params: 8,129
Trainable params: 8,129
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
71/71 [==============================] - ETA: 0s - loss: 0.0557 - mean_squared_error: 0.0557
Epoch 00001: val_loss improved from inf to 0.00870, saving model to SRCNN_check.h5
71/71 [==============================] - 61s 853ms/step - 

71/71 [==============================] - ETA: 0s - loss: 0.0852 - mean_squared_error: 0.0852
Epoch 00001: val_loss improved from inf to 0.00963, saving model to DNCNN_check.h5
71/71 [==============================] - 1260s 18s/step - loss: 0.0852 - mean_squared_error: 0.0852 - val_loss: 0.0096 - val_mean_squared_error: 0.0096
Epoch 2/10
71/71 [==============================] - ETA: 0s - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 00002: val_loss improved from 0.00963 to 0.00843, saving model to DNCNN_check.h5
71/71 [==============================] - 1276s 18s/step - loss: 0.0045 - mean_squared_error: 0.0045 - val_loss: 0.0084 - val_mean_squared_error: 0.0084
Epoch 3/10
71/71 [==============================] - ETA: 0s - loss: 0.0037 - mean_squared_error: 0.0037
Epoch 00003: val_loss improved from 0.00843 to 0.00709, saving model to DNCNN_check.h5
71/71 [==============================] - 1252s 18s/step - loss: 0.0037 - mean_squared_error: 0.0037 - val_loss: 0.0071 - val_mean_squared_

In [23]:
###### import numpy as np
import numpy
import math
#from models import interpolation , SRCNN_train , SRCNN_model, SRCNN_predict , DNCNN_train , DNCNN_model , DNCNN_predict
#from scipy.misc import imresize
from scipy.io import loadmat
import matplotlib.pyplot as plt



if __name__ == "__main__":
    # load datasets 
    channel_model = "VehA"
    SNR = 12
    Number_of_pilots = 48
  #  perfect = loadmat("Perfect_H_40000"+ channel_model.mat)['My_perfect_H']
    #perfect = loadmat("Perfect_H_40000", appendmat = True)['My_perfect_H']
    #noisy_input = loadmat("Noisy_" + channel_model + "_" + "SNR_" + str(SNR) + ".mat") [channel_model+"_noisy_"+ str(SNR)]  
    #noisy_input = loadmat("My_noisy_H_22", appendmat = True )["My_noisy_H"]  
                      
    interp_noisy = interpolation(noisy_input , SNR , Number_of_pilots , 'rbf')

    perfect_image = numpy.zeros((len(perfect),72,14,2))
    perfect_image[:,:,:,0] = numpy.real(perfect)
    perfect_image[:,:,:,1] = numpy.imag(perfect)
    perfect_image = numpy.concatenate((perfect_image[:,:,:,0], perfect_image[:,:,:,1]), axis=0).reshape(2*len(perfect), 72, 14, 1)
    
    
    ####### ------ training SRCNN ------ #######
    idx_random = numpy.random.rand(len(perfect_image)) < (1/9)  # uses 32000 from 36000 as training and the rest as validation
    train_data, train_label = interp_noisy[idx_random,:,:,:] , perfect_image[idx_random,:,:,:]
    val_data, val_label = interp_noisy[~idx_random,:,:,:] , perfect_image[~idx_random,:,:,:]    
    SRCNN_train(train_data ,train_label, val_data , val_label , channel_model , Number_of_pilots , SNR )
    
   
    ####### ------ prediction using SRCNN ------ #######
    srcnn_pred_train = SRCNN_predict(train_data, channel_model , Number_of_pilots , SNR)
    srcnn_pred_validation = SRCNN_predict(val_data, channel_model , Number_of_pilots , SNR)  
                      
                      
    ####### ------ training DNCNN ------ #######
    
    DNCNN_train(train_data ,train_label, val_data , val_label, channel_model , Number_of_pilots, SNR )
    
    ######--------predicting using DNCNN ----####
    
    DNCNN_pred = DNCNN_predict(train_data, channel_model , Number_of_pilots , SNR)

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 72, 14, 1)]       0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 72, 14, 64)        5248      
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 72, 14, 32)        2080      
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 72, 14, 1)         801       
Total params: 8,129
Trainable params: 8,129
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
69/70 [============================>.] - ETA: 0s - loss: 0.1766 - mean_squared_error: 0.1766
Epoch 00001: val_loss improved from inf to 0.01705, saving model to SRCNN_check.h5
70/70 [==============================] - 64s 920ms/step -

70/70 [==============================] - ETA: 0s - loss: 0.0631 - mean_squared_error: 0.0631
Epoch 00001: val_loss improved from inf to 0.00935, saving model to DNCNN_check.h5
70/70 [==============================] - 1273s 18s/step - loss: 0.0631 - mean_squared_error: 0.0631 - val_loss: 0.0094 - val_mean_squared_error: 0.0094
Epoch 2/10
70/70 [==============================] - ETA: 0s - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 00002: val_loss improved from 0.00935 to 0.00888, saving model to DNCNN_check.h5
70/70 [==============================] - 1289s 18s/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0089 - val_mean_squared_error: 0.0089
Epoch 3/10
70/70 [==============================] - ETA: 0s - loss: 0.0035 - mean_squared_error: 0.0035 
Epoch 00003: val_loss improved from 0.00888 to 0.00805, saving model to DNCNN_check.h5
70/70 [==============================] - 1646s 24s/step - loss: 0.0035 - mean_squared_error: 0.0035 - val_loss: 0.0080 - val_mean_squared

In [26]:
###### import numpy as np
import numpy
import math
#from models import interpolation , SRCNN_train , SRCNN_model, SRCNN_predict , DNCNN_train , DNCNN_model , DNCNN_predict
#from scipy.misc import imresize
from scipy.io import loadmat
import matplotlib.pyplot as plt



if __name__ == "__main__":
    # load datasets 
    channel_model = "VehA"
    SNR = 25
    Number_of_pilots = 48
  #  perfect = loadmat("Perfect_H_40000"+ channel_model.mat)['My_perfect_H']
    #perfect = loadmat("Perfect_H_40000", appendmat = True)['My_perfect_H']
    #noisy_input = loadmat("Noisy_" + channel_model + "_" + "SNR_" + str(SNR) + ".mat") [channel_model+"_noisy_"+ str(SNR)]  
    #noisy_input = loadmat("My_noisy_H_22", appendmat = True )["My_noisy_H"]  
                      
    interp_noisy = interpolation(noisy_input , SNR , Number_of_pilots , 'rbf')

    perfect_image = numpy.zeros((len(perfect),72,14,2))
    perfect_image[:,:,:,0] = numpy.real(perfect)
    perfect_image[:,:,:,1] = numpy.imag(perfect)
    perfect_image = numpy.concatenate((perfect_image[:,:,:,0], perfect_image[:,:,:,1]), axis=0).reshape(2*len(perfect), 72, 14, 1)
    
    
    ####### ------ training SRCNN ------ #######
    idx_random = numpy.random.rand(len(perfect_image)) < (1/9)  # uses 32000 from 36000 as training and the rest as validation
    train_data, train_label = interp_noisy[idx_random,:,:,:] , perfect_image[idx_random,:,:,:]
    val_data, val_label = interp_noisy[~idx_random,:,:,:] , perfect_image[~idx_random,:,:,:]    
    SRCNN_train(train_data ,train_label, val_data , val_label , channel_model , Number_of_pilots , SNR )
    
   
    ####### ------ prediction using SRCNN ------ #######
    srcnn_pred_train = SRCNN_predict(train_data, channel_model , Number_of_pilots , SNR)
    srcnn_pred_validation = SRCNN_predict(val_data, channel_model , Number_of_pilots , SNR)  
                      
                      
    ####### ------ training DNCNN ------ #######
    
    DNCNN_train(train_data ,train_label, val_data , val_label, channel_model , Number_of_pilots, SNR )
    
    ######--------predicting using DNCNN ----####
    
    DNCNN_pred = DNCNN_predict(train_data, channel_model , Number_of_pilots , SNR)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 72, 14, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 72, 14, 64)        5248      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 14, 32)        2080      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 14, 1)         801       
Total params: 8,129
Trainable params: 8,129
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
70/70 [==============================] - ETA: 0s - loss: 0.0939 - mean_squared_error: 0.0939
Epoch 00001: val_loss improved from inf to 0.01290, saving model to SRCNN_check.h5
70/70 [==============================] - 106s 2s/step - lo

Epoch 1/10
70/70 [==============================] - ETA: 0s - loss: 0.0709 - mean_squared_error: 0.0709
Epoch 00001: val_loss improved from inf to 0.01229, saving model to DNCNN_check.h5
70/70 [==============================] - 1266s 18s/step - loss: 0.0709 - mean_squared_error: 0.0709 - val_loss: 0.0123 - val_mean_squared_error: 0.0123
Epoch 2/10
70/70 [==============================] - ETA: 0s - loss: 0.0041 - mean_squared_error: 0.0041
Epoch 00002: val_loss improved from 0.01229 to 0.00899, saving model to DNCNN_check.h5
70/70 [==============================] - 1267s 18s/step - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.0090 - val_mean_squared_error: 0.0090
Epoch 3/10
70/70 [==============================] - ETA: 0s - loss: 0.0033 - mean_squared_error: 0.0033
Epoch 00003: val_loss improved from 0.00899 to 0.00831, saving model to DNCNN_check.h5
70/70 [==============================] - 1297s 19s/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0083 - val_me

KeyboardInterrupt: 

In [12]:
###### import numpy as np
import numpy
import math
#from models import interpolation , SRCNN_train , SRCNN_model, SRCNN_predict , DNCNN_train , DNCNN_model , DNCNN_predict
#from scipy.misc import imresize
from scipy.io import loadmat
import matplotlib.pyplot as plt



if __name__ == "__main__":
    # load datasets 
    channel_model = "VehA"
    SNR = 20
    Number_of_pilots = 8
  #  perfect = loadmat("Perfect_H_40000"+ channel_model.mat)['My_perfect_H']
    #perfect = loadmat("Perfect_H_40000", appendmat = True)['My_perfect_H']
    #noisy_input = loadmat("Noisy_" + channel_model + "_" + "SNR_" + str(SNR) + ".mat") [channel_model+"_noisy_"+ str(SNR)]  
    #noisy_input = loadmat("My_noisy_H_22", appendmat = True )["My_noisy_H"]  
                      
    interp_noisy = interpolation(noisy_input , SNR , Number_of_pilots , 'rbf')

    perfect_image = numpy.zeros((len(perfect),72,14,2))
    perfect_image[:,:,:,0] = numpy.real(perfect)
    perfect_image[:,:,:,1] = numpy.imag(perfect)
    perfect_image = numpy.concatenate((perfect_image[:,:,:,0], perfect_image[:,:,:,1]), axis=0).reshape(2*len(perfect), 72, 14, 1)
    
    
    ####### ------ training SRCNN ------ #######
    idx_random = numpy.random.rand(len(perfect_image)) < (1/9)  # uses 32000 from 36000 as training and the rest as validation
    train_data, train_label = interp_noisy[idx_random,:,:,:] , perfect_image[idx_random,:,:,:]
    val_data, val_label = interp_noisy[~idx_random,:,:,:] , perfect_image[~idx_random,:,:,:]    
    SRCNN_train(train_data ,train_label, val_data , val_label , channel_model , Number_of_pilots , SNR )
    
   
    ####### ------ prediction using SRCNN ------ #######
    srcnn_pred_train = SRCNN_predict(train_data, channel_model , Number_of_pilots , SNR)
    srcnn_pred_validation = SRCNN_predict(val_data, channel_model , Number_of_pilots , SNR)  
                      
                      
    ####### ------ training DNCNN ------ #######
    
    DNCNN_train(train_data ,train_label, val_data , val_label, channel_model , Number_of_pilots, SNR )
    
    ######--------predicting using DNCNN ----####
    
    DNCNN_pred = DNCNN_predict(train_data, channel_model , Number_of_pilots , SNR)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 72, 14, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 72, 14, 64)        5248      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 14, 32)        2080      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 14, 1)         801       
Total params: 8,129
Trainable params: 8,129
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
70/70 [==============================] - ETA: 0s - loss: 0.0693 - mean_squared_error: 0.0693
Epoch 00001: val_loss improved from inf to 0.03179, saving model to SRCNN_check.h5
70/70 [==============================] - 99s 1s/step - los

70/70 [==============================] - ETA: 0s - loss: 0.1500 - mean_squared_error: 0.1500
Epoch 00001: val_loss improved from inf to 0.17461, saving model to DNCNN_check.h5
70/70 [==============================] - 1471s 21s/step - loss: 0.1500 - mean_squared_error: 0.1500 - val_loss: 0.1746 - val_mean_squared_error: 0.1746
Epoch 2/5
70/70 [==============================] - ETA: 0s - loss: 0.0250 - mean_squared_error: 0.0250
Epoch 00002: val_loss improved from 0.17461 to 0.15561, saving model to DNCNN_check.h5
70/70 [==============================] - 1415s 20s/step - loss: 0.0250 - mean_squared_error: 0.0250 - val_loss: 0.1556 - val_mean_squared_error: 0.1556
Epoch 3/5
70/70 [==============================] - ETA: 0s - loss: 0.0235 - mean_squared_error: 0.0235
Epoch 00003: val_loss improved from 0.15561 to 0.13532, saving model to DNCNN_check.h5
70/70 [==============================] - 1936s 28s/step - loss: 0.0235 - mean_squared_error: 0.0235 - val_loss: 0.1353 - val_mean_squared_er